In [2]:
from lumibot.brokers import Alpaca
from lumibot.backtesting import YahooDataBacktesting
from lumibot.strategies.strategy import Strategy
from lumibot.traders import Trader
from datetime import datetime
from alpaca_trade_api import REST
from timedelta import Timedelta
import lumibot
import alpaca_trade_api as tradeapi
import pandas as pd
import time
import numpy as np
import matplotlib.pyplot as plt
from datetime import date

from typing import Tuple 
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"
import os

API_KEY = "PK5VAF98LA1ZLKLCZB38"
API_SECRET = "snRgfhRBLSdTvcVLig1kJwOqmBFsmAiQr8T3d8Wm"
BASE_URL = "https://paper-api.alpaca.markets"

# Set the environment variable to disable parallelism
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Now you can import the necessary modules from tokenizers
from transformers import AutoTokenizer, AutoModelForSequenceClassification
api = tradeapi.REST(API_KEY, API_SECRET, base_url=BASE_URL, api_version='v2')





In [3]:

device = "cuda:0" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert").to(device)
labels = ["positive", "negative", "neutral"]

def estimate_sentiment(news):
    if news:
        tokens = tokenizer(news, return_tensors="pt", padding=True).to(device)
        result = model(tokens["input_ids"], attention_mask=tokens["attention_mask"])[
                "logits"
        ]
        result = torch.nn.functional.softmax(torch.sum(result, 0), dim=-1)
        probability = result[torch.argmax(result)]
        sentiment = labels[torch.argmax(result)]
        return probability, sentiment
    else:
        return 0, labels[-1]
    
ALPACA_CREDS = {
    "API_KEY": API_KEY,
    "API_SECRET": API_SECRET,
    "PAPER": True
}

def get_historical_data(symbol, start_date, end_date, timeframe='1D'):
    data = api.get_bars(symbol, timeframe, start=start_date, end=end_date).df
    return data

class MLTrader(Strategy): 
    
    def initialize(self, symbol:str="SPY", cash_at_risk:float=.5): 
        self.symbol = symbol
        #self.symbols = ["SPY","..."]
        self.sleeptime = "24H" 
        self.last_trade = None 
        self.cash_at_risk = cash_at_risk
        self.api = REST(base_url=BASE_URL, key_id=API_KEY, secret_key=API_SECRET)

    def position_sizing(self): 
        cash = self.get_cash() 
        last_price = self.get_last_price(self.symbol)
        quantity = round(cash * self.cash_at_risk / last_price,0)
        return cash, last_price, quantity

    def get_dates(self): 
        today = self.get_datetime()
        three_days_prior = today - Timedelta(days=3)
        return today.strftime('%Y-%m-%d'), three_days_prior.strftime('%Y-%m-%d')
    
    # Method for blank days prior where we have a variable day amount
    def get_dates_var(self,days):
        today = self.get_datetime()
        days_prior = today - Timedelta(days=days)
        return today.strftime('%Y-%m-%d'), days_prior.strftime('%Y-%m-%d')

    def get_sentiment(self): 
        today, three_days_prior = self.get_dates()
        news = self.api.get_news(symbol=self.symbol, 
                                 start=three_days_prior, 
                                 end=today) 
        news = [ev.__dict__["_raw"]["headline"] for ev in news]
        probability, sentiment = estimate_sentiment(news)
        return probability, sentiment 
    
    '''def trading_equation(self):
        buy_signal = False
        sell_signal = False
        end_date , start_date = self.get_dates2week()
        data = api.get_bars("SPY", '1D', start=start_date, end=end_date).df
    
        return data'''
    
    def get_historical_data(symbol, start_date, end_date, timeframe):
        data = api.get_bars(symbol, timeframe, start=start_date, end=end_date).df
        print(data)
        return data
        
    
    '''def calculate_bollinger_bands(data):
        std_multiplier =2
        window=20
        data['Middle_Band'] = data['close'].rolling(window=window).mean()
        data['Upper_Band'] = data['Middle_Band'] + (data['close'].rolling(window=window).std() * std_multiplier)
        data['Lower_Band'] = data['Middle_Band'] - (data['close'].rolling(window=window).std() * std_multiplier)
        return data
    
    def identify_trend(data, short_window=50, long_window=200):
        data = data.copy()  # Make a copy 

        data['Short_MA'] = data['close'].rolling(window=short_window).mean()
        data['Long_MA'] = data['close'].rolling(window=long_window).mean()
        data.dropna(inplace=True)

        data['Signal'] = 0  # 0: No Signal, 1: Buy Signal, -1: Sell Signal
        data['Signal'][(data['close'] < data['Lower_Band']) & (data['Short_MA'] / data['Long_MA'] > 0.75)] = -1
        data['Signal'][(data['close'] > data['Upper_Band']) & (data['Short_MA'] / data['Long_MA'] < 0.25)] = 1

        return data '''
    
    def plot_trend(self, data, short_window, long_window):
        # Calculate moving averages
        data['Short_MA'] = data['close'].rolling(window=short_window).mean()
        data['Long_MA'] = data['close'].rolling(window=long_window).mean()

        # Calculate Bollinger Bands
        std_multiplier = 2
        data['Middle_Band'] = data['close'].rolling(window=20).mean()
        data['Upper_Band'] = data['Middle_Band'] + (data['close'].rolling(window=20).std() * std_multiplier)
        data['Lower_Band'] = data['Middle_Band'] - (data['close'].rolling(window=20).std() * std_multiplier)

        # Plot the trend
        plt.figure(figsize=(12, 6))
        plt.plot(data.index, data['close'], label='Close Price', color='blue')
        plt.plot(data.index, data['Short_MA'], label=f'Short MA ({short_window})', color='orange')
        plt.plot(data.index, data['Long_MA'], label=f'Long MA ({long_window})', color='green')
        plt.plot(data.index, data['Middle_Band'], label='Middle Band', color='orange')
        plt.plot(data.index, data['Upper_Band'], label='Upper Band', color='green')
        plt.plot(data.index, data['Lower_Band'], label='Lower Band', color='red')
    
        plt.title('Stock Price and Trend Analysis')
        plt.xlabel('Date')
        plt.ylabel('Close Price')
        plt.legend()
        plt.show()


    def on_trading_iteration(self):
        short_window = 50
        long_window = 200
        symbol = "SPY"
        cash, last_price, quantity = self.position_sizing() 
        probability, sentiment = self.get_sentiment()
        
        end_date,start_date = self.get_dates_var(20)
        data = get_historical_data(symbol, start_date, end_date, '1D')
        self.plot_trend(data, short_window, long_window)
        
        #data = self.trading_equation()
        #dataTrend = self.identify_trend(data, short_window, long_window)
        

        
        #bars = self.get_historical_prices("SPY", 2, "day")
        #df = bars.df
        #self.log_message(df)
        
        
           
        if cash > last_price: 
            if sentiment == "positive" and probability > .999: 
                if self.last_trade == "sell": 
                    self.sell_all() 
                order = self.create_order(
                    self.symbol, 
                    quantity, 
                    "buy", 
                    type="bracket", 
                    take_profit_price=last_price*1.20, 
                    stop_loss_price=last_price*.95
                )
                self.submit_order(order) 
                self.last_trade = "buy"
            elif sentiment == "negative" and probability > .999: 
                if self.last_trade == "buy": 
                    self.sell_all() 
                order = self.create_order(
                    self.symbol, 
                    quantity, 
                    "sell", 
                    type="bracket", 
                    take_profit_price=last_price*.8, 
                    stop_loss_price=last_price*1.05
                )
                self.submit_order(order) 
                self.last_trade = "sell"


start_date = datetime(2020,1,1)
end_date = datetime(2023,12,31) 
broker = Alpaca(ALPACA_CREDS) 
strategy = MLTrader(name='mlstrat', broker=broker, 
                    parameters={"symbol":"SPY", 
                                "cash_at_risk":.5})
strategy.backtest(
    YahooDataBacktesting, 
    start_date, 
    end_date, 
    parameters={"symbol":"SPY", "cash_at_risk":.5}
)

APIError: {"message": "forbidden."}


ERROR:alpaca.trading.stream:error during websocket communication: failed to authenticate
Traceback (most recent call last):
  File "/Users/rohanchintham/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 170, in _run_forever
    await self._start_ws()
  File "/Users/rohanchintham/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 131, in _start_ws
    await self._auth()
  File "/Users/rohanchintham/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 78, in _auth
    raise ValueError("failed to authenticate")
ValueError: failed to authenticate
ERROR:alpaca.trading.stream:error during websocket communication: failed to authenticate
Traceback (most recent call last):
  File "/Users/rohanchintham/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 170, in _run_forever
    await self._start_ws()
  File "/Users/rohanchintham/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 131, in _start_ws
  

ERROR:alpaca.trading.stream:error during websocket communication: failed to authenticate
Traceback (most recent call last):
  File "/Users/rohanchintham/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 170, in _run_forever
    await self._start_ws()
  File "/Users/rohanchintham/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 131, in _start_ws
    await self._auth()
  File "/Users/rohanchintham/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 78, in _auth
    raise ValueError("failed to authenticate")
ValueError: failed to authenticate
ERROR:alpaca.trading.stream:error during websocket communication: failed to authenticate
Traceback (most recent call last):
  File "/Users/rohanchintham/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 170, in _run_forever
    await self._start_ws()
  File "/Users/rohanchintham/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 131, in _start_ws
  

ERROR:alpaca.trading.stream:error during websocket communication: failed to authenticate
Traceback (most recent call last):
  File "/Users/rohanchintham/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 170, in _run_forever
    await self._start_ws()
  File "/Users/rohanchintham/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 131, in _start_ws
    await self._auth()
  File "/Users/rohanchintham/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 78, in _auth
    raise ValueError("failed to authenticate")
ValueError: failed to authenticate
ERROR:alpaca.trading.stream:error during websocket communication: failed to authenticate
Traceback (most recent call last):
  File "/Users/rohanchintham/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 170, in _run_forever
    await self._start_ws()
  File "/Users/rohanchintham/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 131, in _start_ws
  

ERROR:alpaca.trading.stream:error during websocket communication: failed to authenticate
Traceback (most recent call last):
  File "/Users/rohanchintham/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 170, in _run_forever
    await self._start_ws()
  File "/Users/rohanchintham/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 131, in _start_ws
    await self._auth()
  File "/Users/rohanchintham/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 78, in _auth
    raise ValueError("failed to authenticate")
ValueError: failed to authenticate
ERROR:alpaca.trading.stream:error during websocket communication: failed to authenticate
Traceback (most recent call last):
  File "/Users/rohanchintham/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 170, in _run_forever
    await self._start_ws()
  File "/Users/rohanchintham/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 131, in _start_ws
  

...

ERROR:alpaca.trading.stream:error during websocket communication: [Errno 8] nodename nor servname provided, or not known
Traceback (most recent call last):
  File "/Users/rohanchintham/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 170, in _run_forever
    await self._start_ws()
  File "/Users/rohanchintham/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 130, in _start_ws
    await self._connect()
  File "/Users/rohanchintham/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 61, in _connect
    self._ws = await websockets.connect(self._endpoint, **self._websocket_params)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/rohanchintham/anaconda3/lib/python3.11/site-packages/websockets/legacy/client.py", line 655, in __await_impl_timeout__
    return await self.__await_impl__()
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/rohanchintham/anaconda3/lib/python3.11/site-p

ERROR:alpaca.trading.stream:error during websocket communication: [Errno 8] nodename nor servname provided, or not known
Traceback (most recent call last):
  File "/Users/rohanchintham/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 170, in _run_forever
    await self._start_ws()
  File "/Users/rohanchintham/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 130, in _start_ws
    await self._connect()
  File "/Users/rohanchintham/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 61, in _connect
    self._ws = await websockets.connect(self._endpoint, **self._websocket_params)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/rohanchintham/anaconda3/lib/python3.11/site-packages/websockets/legacy/client.py", line 655, in __await_impl_timeout__
    return await self.__await_impl__()
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/rohanchintham/anaconda3/lib/python3.11/site-p

ERROR:alpaca.trading.stream:error during websocket communication: [Errno 8] nodename nor servname provided, or not known
Traceback (most recent call last):
  File "/Users/rohanchintham/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 170, in _run_forever
    await self._start_ws()
  File "/Users/rohanchintham/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 130, in _start_ws
    await self._connect()
  File "/Users/rohanchintham/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 61, in _connect
    self._ws = await websockets.connect(self._endpoint, **self._websocket_params)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/rohanchintham/anaconda3/lib/python3.11/site-packages/websockets/legacy/client.py", line 655, in __await_impl_timeout__
    return await self.__await_impl__()
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/rohanchintham/anaconda3/lib/python3.11/site-p

ERROR:alpaca.trading.stream:error during websocket communication: [Errno 8] nodename nor servname provided, or not known
Traceback (most recent call last):
  File "/Users/rohanchintham/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 170, in _run_forever
    await self._start_ws()
  File "/Users/rohanchintham/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 130, in _start_ws
    await self._connect()
  File "/Users/rohanchintham/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 61, in _connect
    self._ws = await websockets.connect(self._endpoint, **self._websocket_params)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/rohanchintham/anaconda3/lib/python3.11/site-packages/websockets/legacy/client.py", line 655, in __await_impl_timeout__
    return await self.__await_impl__()
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/rohanchintham/anaconda3/lib/python3.11/site-p

ERROR:alpaca.trading.stream:error during websocket communication: [Errno 8] nodename nor servname provided, or not known
Traceback (most recent call last):
  File "/Users/rohanchintham/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 170, in _run_forever
    await self._start_ws()
  File "/Users/rohanchintham/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 130, in _start_ws
    await self._connect()
  File "/Users/rohanchintham/anaconda3/lib/python3.11/site-packages/alpaca/trading/stream.py", line 61, in _connect
    self._ws = await websockets.connect(self._endpoint, **self._websocket_params)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/rohanchintham/anaconda3/lib/python3.11/site-packages/websockets/legacy/client.py", line 655, in __await_impl_timeout__
    return await self.__await_impl__()
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/rohanchintham/anaconda3/lib/python3.11/site-p